# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
# Import API key
from api_keys import geoapify_key



In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../starter_code/output_data/cities.csv")

# Display sample data

city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gulu,2.7746,32.2990,64.18,62,15,3.20,UG,1669944618
1,1,albany,42.6001,-73.9662,35.64,63,97,1.01,US,1669944474
2,2,bengkulu,-3.8004,102.2655,79.11,80,100,4.03,ID,1669944619
3,3,mokhsogollokh,61.4681,128.9203,-19.25,94,100,3.24,RU,1669944619
4,4,busselton,-33.6500,115.3333,67.57,54,0,2.21,AU,1669944322


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)

fig=gmaps.figure(())
# Display the map
fig


Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_narrow=city_data_df.loc[(city_data_df['Max Temp'] <80)&
                         (city_data_df['Max Temp'] >70)&        
                        (city_data_df['Humidity']>20)&
                        (city_data_df['Cloudiness'] ==0)&
                        (city_data_df['Wind Speed'] <10)]
# Drop any rows with null values
city_data_df_narrow = city_data_df_narrow.dropna()

# Display sample data
city_data_df_narrow


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,97,mangrol,21.1167,70.1167,71.31,57,0,7.74,IN,1669944663
107,107,broken hill,-31.9500,141.4333,79.92,26,0,7.96,AU,1669944668
124,124,minna,9.6139,6.5569,74.91,50,0,3.44,NG,1669944676
126,126,beira,-19.8436,34.8389,71.69,94,0,4.61,MZ,1669944514
166,166,cabo san lucas,22.8909,-109.9124,78.10,78,0,5.75,MX,1669944695
460,460,salalah,17.0151,54.0924,71.69,73,0,1.14,OM,1669944649
517,517,berbera,10.4396,45.0143,77.07,80,0,3.24,SO,1669944848
552,552,pedernales,18.0384,-71.7440,78.19,72,0,4.97,DO,1669944864


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, 
# and humidity
hotel_df= city_data_df_narrow[["City", "Country", "Max Temp", "Lat", "Lng",'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
97,mangrol,IN,71.31,21.1167,70.1167,57,
107,broken hill,AU,79.92,-31.9500,141.4333,26,
124,minna,NG,74.91,9.6139,6.5569,50,
126,beira,MZ,71.69,-19.8436,34.8389,94,
166,cabo san lucas,MX,78.10,22.8909,-109.9124,78,
460,salalah,OM,71.69,17.0151,54.0924,73,
517,berbera,SO,77.07,10.4396,45.0143,80,
552,pedernales,DO,78.19,18.0384,-71.7440,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius =10000
params = { 'radius':radius,
            'type':"lodging",
              'apiKey':geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    Lat= row['Lat']
    Lng= row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{Lng},{Lat}'
    params["bias"] = f'proximity:{Lng},{Lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    hotel_request = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address =  hotel_request.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mangrol - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
minna - nearest hotel: No hotel found
beira - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
berbera - nearest hotel: No hotel found
pedernales - nearest hotel: No hotel found


,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
97,mangrol,IN,71.31,21.1167,70.1167,57,No hotel found
107,broken hill,AU,79.92,-31.9500,141.4333,26,No hotel found
124,minna,NG,74.91,9.6139,6.5569,50,No hotel found
126,beira,MZ,71.69,-19.8436,34.8389,94,No hotel found
166,cabo san lucas,MX,78.10,22.8909,-109.9124,78,No hotel found
460,salalah,OM,71.69,17.0151,54.0924,73,No hotel found
517,berbera,SO,77.07,10.4396,45.0143,80,No hotel found
552,pedernales,DO,78.19,18.0384,-71.7440,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)